# 01 – Data Cleaning and Synthetic Data Generation

This notebook creates synthetic data for a subscription-based SaaS company.  
It produces four base tables:

- customers  
- subscription plans  
- customer subscriptions  
- monthly subscription activity  

All outputs are written into `data/raw/` and `data/processed/` for use in SQL scripts, analysis notebooks, and dashboards.



In [ ]:
import numpy as np
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import os

# File paths (simple, no external dependencies)
RAW_DIR = "data/raw"
PROC_DIR = "data/processed"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)

# Analysis window for generated activity
START_DATE = date(2022, 1, 1)
END_DATE = date(2025, 10, 31)

np.random.seed(42)


## Plans table

Defines the subscription plans offered by the SaaS product.

In [ ]:
# Generate subscription plans

plans = [
    ("Basic",    "monthly", 29.0),
    ("Standard", "monthly", 59.0),
    ("Premium",  "monthly", 119.0),
    ("Basic",    "annual", 299.0),
    ("Standard", "annual", 599.0),
    ("Premium",  "annual", 1199.0),
]

plans_df = pd.DataFrame(plans, columns=["plan_name", "billing_period", "price_usd"])
plans_df["plan_id"] = range(1, len(plans_df) + 1)
plans_df = plans_df[["plan_id", "plan_name", "billing_period", "price_usd"]]

plans_df.to_csv(f"{RAW_DIR}/plans.csv", index=False)
plans_df.head()

,plan_id,plan_name,billing_period,price_usd
0,1,Basic,monthly,29.0
1,2,Standard,monthly,59.0
2,3,Premium,monthly,119.0
3,4,Basic,annual,299.0
4,5,Standard,annual,599.0


## Customers table

Represents individual customers of the SaaS product.


In [ ]:
# Generate customers with mild growth over time

n_customers = 8000

countries = ["US", "UK", "CA", "DE", "FR", "NL", "AU", "IN", "BR", "SE"]
channels = ["Organic", "Paid Ads", "Referral", "Email", "Partnership"]
devices = ["desktop", "mobile"]

# Generate signup dates with increasing density over time
days_total = (END_DATE - START_DATE).days
growth_weights = np.linspace(0.5, 1.5, days_total + 1)
growth_weights /= growth_weights.sum()

date_range = pd.date_range(START_DATE, END_DATE)
signup_dates = np.random.choice(
    date_range,
    size=n_customers,
    p=growth_weights
)

customers_df = pd.DataFrame({
    "customer_id": range(1, n_customers + 1),
    "country": np.random.choice(countries, size=n_customers),
    "acquisition_channel": np.random.choice(
        channels, size=n_customers,
        p=[0.35, 0.25, 0.15, 0.15, 0.10]
    ),
    "device_type": np.random.choice(
        devices, size=n_customers,
        p=[0.45, 0.55]
    ),
    "signup_date": signup_dates
})

customers_df["signup_date"] = customers_df["signup_date"].dt.date

customers_df.to_csv(f"{RAW_DIR}/customers.csv", index=False)
customers_df.head()


,customer_id,country,acquisition_channel,device_type,signup_date
0,1,UK,Organic,mobile,2023-12-01
1,2,SE,Organic,mobile,2025-09-15
2,3,US,Paid Ads,mobile,2025-02-06
3,4,SE,Partnership,desktop,2024-09-11
4,5,IN,Partnership,desktop,2022-12-16


## Subscriptions table

Each row represents a customer's subscription to a specific plan.
Includes start dates, cancellation behaviour, and MRR.


In [ ]:
# Generate subscriptions with realistic churn probability model

def choose_num_subs():
    return np.random.choice([0, 1, 2], p=[0.40, 0.50, 0.10])

subscriptions = []
sub_id = 1

plans_by_id = plans_df.set_index("plan_id")

# Churn probabilities
prob_churn_month_1 = 0.10   # 10% in month 1
prob_churn_later = 0.03     # 3% per month after

for _, cust in customers_df.iterrows():
    num_subs = choose_num_subs()
    if num_subs == 0:
        continue

    possible_plan_ids = plans_df["plan_id"].values
    selected_ids = np.random.choice(possible_plan_ids, size=num_subs, replace=False)

    for plan_id in selected_ids:
        plan = plans_by_id.loc[plan_id]

        # Subscription start — short delay after signup
        start_offset = np.random.randint(0, 60)
        start_dt = pd.Timestamp(cust["signup_date"]) + pd.Timedelta(days=int(start_offset))
        start_date = start_dt.date()

        if start_date > END_DATE:
            continue

        # MRR
        mrr = plan["price_usd"] if plan["billing_period"] == "monthly" else plan["price_usd"] / 12.0
        mrr = round(mrr, 2)

        # Realistic churn walk
        current_date = start_date
        month_index = 0
        cancel_date = None

        while current_date <= END_DATE:
            churn_prob = prob_churn_month_1 if month_index == 0 else prob_churn_later
            if np.random.rand() < churn_prob:
                cancel_date = current_date
                break

            current_date = (pd.Timestamp(current_date) + relativedelta(months=1)).date()
            month_index += 1

        status = "cancelled" if cancel_date else "active"

        subscriptions.append({
            "subscription_id": sub_id,
            "customer_id": int(cust["customer_id"]),
            "plan_id": int(plan_id),
            "start_date": start_date,
            "cancel_date": cancel_date,
            "status": status,
            "mrr_usd": mrr
        })

        sub_id += 1

subscriptions_df = pd.DataFrame(subscriptions)

subscriptions_df.to_csv(f"{RAW_DIR}/subscriptions.csv", index=False)
subscriptions_df.head()



,subscription_id,customer_id,plan_id,start_date,cancel_date,status,mrr_usd
0,1,5,4,2023-01-02,2023-12-02,cancelled,24.92
1,2,7,6,2022-06-29,2024-07-28,cancelled,99.92
2,3,8,6,2025-08-05,None,active,99.92
3,4,8,2,2025-08-07,None,active,59.00
4,5,9,5,2024-11-09,None,active,49.92


## Monthly subscription activity

Creates one row per subscription for each month it is active.
Used for cohort analysis, retention curves, churn, MRR, and forecasting.


In [ ]:
# Generate monthly subscription activity

def month_range(start, end):
    """Return a list of YYYY-MM-01 dates from start to end inclusive."""
    current = date(start.year, start.month, 1)
    end_month = date(end.year, end.month, 1)
    months = []
    while current <= end_month:
        months.append(current)
        current = (pd.Timestamp(current) + relativedelta(months=1)).date()
    return months

activity_rows = []

for _, sub in subscriptions_df.iterrows():
    start = sub["start_date"]
    cancel = sub["cancel_date"]

    months = month_range(start, END_DATE)
    cohort_month = date(start.year, start.month, 1)

    churn_month = date(cancel.year, cancel.month, 1) if pd.notnull(cancel) else None

    for i, m in enumerate(months):
        is_new = (i == 0)
        is_active = (churn_month is None) or (m < churn_month)
        churned_this_month = (churn_month is not None and m == churn_month)

        if churned_this_month:
            mrr_value = sub["mrr_usd"]
        elif is_active:
            mrr_value = sub["mrr_usd"]
        else:
            mrr_value = 0.0

        activity_rows.append({
            "subscription_id": sub["subscription_id"],
            "customer_id": sub["customer_id"],
            "plan_id": sub["plan_id"],
            "month": m,
            "cohort_month": cohort_month,
            "is_active": is_active,
            "is_new": is_new,
            "churned_this_month": churned_this_month,
            "mrr_usd": mrr_value
        })

monthly_activity_df = pd.DataFrame(activity_rows)
monthly_activity_df.to_csv(f"{PROC_DIR}/monthly_activity.csv", index=False)
monthly_activity_df.head()




,subscription_id,customer_id,plan_id,month,cohort_month,is_active,is_new,churned_this_month,mrr_usd
0,1,5,4,2023-01-01,2023-01-01,True,True,False,24.92
1,1,5,4,2023-02-01,2023-01-01,True,False,False,24.92
2,1,5,4,2023-03-01,2023-01-01,True,False,False,24.92
3,1,5,4,2023-04-01,2023-01-01,True,False,False,24.92
4,1,5,4,2023-05-01,2023-01-01,True,False,False,24.92


## Basic sanity checks


In [ ]:
print("Customers:", len(customers_df))
print("Subscriptions:", len(subscriptions_df))
print("Monthly activity rows:", len(monthly_activity_df))

print("\nSubscriptions by status:")
print(subscriptions_df["status"].value_counts())

print("\nMRR checks:")
print("Total active MRR:", monthly_activity_df[monthly_activity_df["is_active"]]["mrr_usd"].sum())
print("Total churned MRR events:", monthly_activity_df[monthly_activity_df["churned_this_month"]]["mrr_usd"].sum())

print("\nChurn sanity:")
print("Rows with churned_this_month == True:", monthly_activity_df["churned_this_month"].sum())
print("Example churn rows:")
print(monthly_activity_df[monthly_activity_df["churned_this_month"]].head())



Customers: 8000
Subscriptions: 5453
Monthly activity rows: 105545

Subscriptions by status:
status
active       3003
cancelled    2450
Name: count, dtype: int64

MRR checks:
Total active MRR: 4267695.719999999
Total churned MRR events: 153989.48

Churn sanity:
Rows with churned_this_month == True: 2450
Example churn rows:
     subscription_id  customer_id  plan_id       month cohort_month  \
11                 1            5        4  2023-12-01   2023-01-01   
59                 2            7        6  2024-07-01   2022-06-01   
113                6           11        4  2023-11-01   2022-03-01   
144                7           14        1  2023-11-01   2023-04-01   
180                8           15        4  2024-03-01   2023-03-01   

     is_active  is_new  churned_this_month  mrr_usd  
11       False   False                True    24.92  
59       False   False                True    99.92  
113      False   False                True    24.92  
144      False   False           